# Embeddings con LSTM



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

AMLO
1. Asegura AMLO que en campaña todo será amor y paz , que los otros candidatos se ahorren sus provocaciones .
377. “ Y tiene que ver con la alianza de el PRI y de el PAN , no hay que olvidar que los de el PRI , los de el PAN , los de el PRIAN se pusieron de acuerdo para nombrar a los consejeros de el INE , que tampoco son blancas palomas y también se pusieron de acuerdo para nombrar a los magistrados de el Tribunal_Electoral , yo lo denuncié en su momento ” , comentó .
753. “ Antes en las elecciones pasadas no podíamos defender nos frente_a los ataques a la guerra sucia , a las calumnias , porque nos cercaban , nos daban espacios o actuaban de manera tendenciosa en los medios convencionales , pero en esta elección la diferencia la están marcando las redes sociales , ya no pueden ” , expresó .
1129. En otro orden de ideas , informó que si hoy fuesen las elecciones ganaría con 25 puntos de ventaja , es bastante el número de mexicanos que lo apoyan , pero hay quienes crearán de maner

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 1000
sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::k//10]]))

y se comprometió a regresar como presidente electo para traer el plan de desarrollo para uruapan para decir cuánto se invertirá en la región y regresará cada seis meses para evaluar el plan .
y es que , dijo , a el debate de el pasado_domingo lópez_obrador se presentó como un hombre profundamente autoritario e incongruente , que desconfía de la sociedad civil y que no tiene un compromiso con el cambio de régimen .
“ vamos a trabajar junto_a cada uno de los mexicanos hasta asegurar les , a cada uno y a cada una , que sus derechos y las oportunidades que se les deben van a estar ahí ” , prometió en su cierre de campaña en saltillo , coahuila , donde dio por concluidas sus actividades proselitistas por las 32 entidades federativas en_busca_de el voto popular .
estas adhesiones involucran a mujeres honestas y decididas , que tienen causas en todos los ámbitos de su vida diaria , y que trabajan por el acceso igualitario a la cultura y cuidado de el medio ambiente sostenible , el respeto y r

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [12]:
freq = sum([Counter(sent) for sent in sentences], Counter())
print(freq.most_common()[::209])

[(',', 2878), ('pan', 19), ('pobres', 9), ('cuidar', 6), ('centros', 5), ('pusieron', 4), ('ven', 3), ('copia', 2), ('ganaderos', 2), ('exterior', 2), ('dejado', 2), ('consistirá', 2), ('dañino', 1), ('llamando', 1), ('descomponiendo', 1), ('secuestrado', 1), ('la_montaña', 1), ('secretaria', 1), ('proselitismo', 1), ('zavala', 1), ('propuesto', 1), ('intelectual', 1), ('sintetizó', 1), ('elabore', 1), ('hacían', 1)]


In [13]:
print(f'Número de tipos: {len(freq.keys())}')

Número de tipos: 5202


Agrego un diccionario para pasar de palabra a índice numérico

In [14]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [15]:
vocab_size = len(w_to_index)
vocab_size

2314

In [16]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = vocab_size
ixEOS = vocab_size + 1
ixUNK = vocab_size + 2

In [17]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Calculo el nuevo tamaño del vocabulario después de agregar 3 tokens

In [18]:
vocab_size = len(w_to_index)
vocab_size

2317

Creo el diccionario inverso, para convertir de índices a palabras

In [19]:
index_to_w = [ w for w, ix in w_to_index.items() ]

Indexo todo el corpus

In [20]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [21]:
sentences_ix = [
    # Le agrego el inicio y fin de caracter a los enunciados
    [ixBOS] + [ w_to_index_unk(w) for w in sent ] + [ixEOS] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([2314,    6,   10,  166,    5,  508,   29,   61,  277,   12,  697,
          3,  178,    1,   77,   12, 1462,   12,  137, 1053,   10,  579,
          8,    4,  298,    6,  509,  167,  405,  451,   12, 2316,    3,
        178,    7]), array([2314, 1054,    2,   27,  840,    4, 1463,    1,    4, 2316,    5,
          3,  580,    6,    1,    3, 2316,    5,    3,  580,    1, 2316,
        168,    0,    2,   11,   16,    1, 2316,    0,  210,    1, 2316,
          7]), array([2314,    8,    3, 2316,    0,    5,  841, 1055, 2316, 2316,    1,
         49,    0, 1056,    3,  162,    6,  169,    2,    9,  147,    5,
          9,  698,  842,    1,   99,   85,  843,    0,  159,    3,    1,
        343,    6,  581,    7])]
Ejemplos Y
[array([   6,   10,  166,    5,  508,   29,   61,  277,   12,  697,    3,
        178,    1,   77,   12, 1462,   12,  137, 1053,   10,  579,    8,
          4,  298,    6,  509,  167,  405,  451,   12, 2316,    3,  178,
          7, 2315]), array([105

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

In [23]:
print(len(X_train))
print(len(X_test))

700
300


Mando los vectores de entrada y salida a tensores en gpu

In [24]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [25]:
X_train = to_pytorch_tensor(X_train)
Y_train = to_pytorch_tensor(Y_train)

X_test = to_pytorch_tensor(X_test)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [43]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 64 # 32

# Dimensión de la capa lstm
D_lstm = 16 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 10

# Learning rate
lr =  0.01

In [44]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True)#, batch_first=True)
        # self.out_layer = nn.Sequential(
        #     nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb, padding_idx=0),
        #     nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True, batch_first=True),
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=D_out)
        # )
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out, bias=True)
        # self.out_layer = nn.Softmax(dim=1)
        # self.out_layer = nn.Sequential(
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=1)
        # )

    def forward(self, sentence):
        T = len(sentence)
        # print(sentence)
        # print(T)

        embeddings = self.embedding(sentence).view(T, 1, -1)
        # print('Embeddings')
        # print(embeddings)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        # print('LSTM_out')
        # print(lstm_out)

        preact_out = self.linear(lstm_out).view(T, -1)
        # print('Preact out')
        # print(preact_out)

        return F.log_softmax(preact_out, dim=1)
        # return self.out_layer(preact_out)

In [45]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(2317, 64)
  (lstm): LSTM(64, 16)
  (linear): Linear(in_features=16, out_features=2317, bias=True)
)

In [46]:
# Inicialización de pesos
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Embedding') != -1:
#         # Regularizo los pesos
#         n = m.num_embeddings
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#     elif classname.find('Linear') != -1:
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)
#     elif classname.find('LSTM') != -1:
#         n = m.input_size
#         y = 1.0/np.sqrt(n)
#         m.weight_ih_l0.data.uniform_(-y, y)
#         m.weight_hh_l0.data.uniform_(-y, y)
#         m.bias_ih_l0.data.fill_(0)
#         m.bias_hh_l0.data.fill_(0)

In [47]:
# model.apply(weights_init)

Entropía cruzada como optmizador y SGD como optimizador

In [48]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)
# criterion = nn.NLLLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

### Entrenamiento

In [51]:
%%time
# for epoch in tqdm(range(num_epochs)):
for epoch in range(num_epochs):
    epoch_loss = 0
    for x, y in zip(X_train, Y_train):
    # for x, y in tqdm(zip(X_train, Y_train)):
        if len(x) == 0:
            tqdm.write('Sentencia vacía')
            continue
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        # print(x)
        pred = model(x)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        # break
    # tqdm.write(f'Epoch: {epoch:>12} Loss: {loss}')
    # break
    print(f'Coste después de {epoch+1} épocas: {epoch_loss}')

Coste después de 1 épocas: 5217.728430747986
Coste después de 2 épocas: 5008.235581874847
Coste después de 3 épocas: 4676.206296920776
Coste después de 4 épocas: 4311.251428842545
Coste después de 5 épocas: 4079.926919937134
Coste después de 6 épocas: 3967.189257144928
Coste después de 7 épocas: 3903.750823497772
Coste después de 8 épocas: 3858.466251373291
Coste después de 9 épocas: 3823.2526485919952
Coste después de 10 épocas: 3794.748399734497
Wall time: 38.6 s


In [40]:
# list(model.named_parameters())

In [41]:
def print_ix_sentence(sentence):
    print(' '.join(index_to_w[ix] for ix in sentence.data))

In [54]:
# See what the scores are after training
with torch.no_grad():
    for sentence, y in zip(X_test[:10], Y_test[:10]):
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()

        out = model(sentence)
        prediccion = torch.argmax(out, dim=1)
        print('- Predicción:')
        print_ix_sentence(prediccion)

        print('- Real:')
        print_ix_sentence(y)
        print('\n')

- Probando enunciado:
<BOS> añadió que <UNK> <UNK> de los funcionarios en diferentes niveles de gobierno , <UNK> y federal , para <UNK> <UNK> la ley , por lo que hizo un llamado a las autoridades para investigar a todos los <UNK> en el <UNK> .

- Predicción:
<UNK> <UNK> <UNK> , , <UNK> , , <UNK> <UNK> <UNK> el <UNK> <UNK> , <UNK> <UNK> <UNK> <UNK> , , <UNK> , <UNK> <UNK> <UNK> <UNK> , , <UNK> <UNK> <UNK> <UNK> , <UNK> <UNK> <UNK> , , <UNK> <UNK> , <EOS>
- Real:
añadió que <UNK> <UNK> de los funcionarios en diferentes niveles de gobierno , <UNK> y federal , para <UNK> <UNK> la ley , por lo que hizo un llamado a las autoridades para investigar a todos los <UNK> en el <UNK> . <EOS>
- Probando enunciado:
<BOS> anuncia que mañana dará_a_conocer el programa de gira de la fase que será presidente electo .

- Predicción:
<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> el <UNK> el <UNK> , <UNK> <UNK> , <UNK> <EOS>
- Real:
anuncia que mañana dará_a_conocer el programa de gira de la fase que será presi